# Introduction
The purpose of this file is to get the public transit data from TDX. Since there are multiple steps to get the complete bilateral public travel data, we figured the form of notebook gives clearer deomnstration of the entire process than python script files.

Specifically, there are three steps:
1. Fetch the data from TDX for all of the bilateral pairs (pairs are split into several files die to API service limitation), and stack the results.
2. Check for missing travel times on either going back or forth, should there exist such pairs, collect them and run again with different departure times.
3. There will still be some pairs with no entry, and they are treated in two ways: fill with walking time (if their walking time is less than 30 mins) or manually look them up using Google Maps (there won't be many so this is viable).

Finally, merge all the data and we will get the public travel time data. Optionally, we can turn the result into a matrix.

# 1. Retrieve Data From TDX
The routing service can only be access through API calls, and we will roughly introduce the process.
First, we acquire an access token, which last for 24 hours, after granted authorization (of accessing API) with our client ID and client secret. Then when we access the routing result through API, with this access token attached.
In our implementation, for the purpose of data storage efficiency, with a pair of points A and B, we fetch the results of "A to B" and "B to A" at the same time and store them in the same row.
Therefore, for a pair of points, the code would be simply calling functions like the following block.

In [ ]:
# Single Pair Example
import TDX_retriever as tr
import pandas as pd

# your info of api
client_id = "your-client-id"
client_secret = "your-client-secret"
TDX = tr.TDX_retriever(client_id, client_secret)

# some centroids as example
c1 = [24.9788580602204, 121.55598430669878]
c2 = [24.92583789587648, 121.34128216256848]

# get routing result from TDX, the returned data is filtered, not original respond
single_pair_res = TDX.get_transport_result(c1, c2)

# save as a dataframe
cols = [
    'A_lat', 'A_lon', 'B_lat', 'B_lon',
    'AB_travel_time', 'AB_ttl_cost', 'AB_transfer_cnt', 'AB_route',
    'BA_travel_time', 'BA_ttl_cost', 'BA_transfer_cnt', 'BA_route'
]
df = pd.DataFrame(columns=cols)
df.loc[len(df)] = single_pair_res

# show the df
df

For a list of pairs, we call the functions in basically the same way. However, due to the API access limitation, we have to split the list into multiple sub-files. Therefore, when running the code, we need to provide the serial number of the sub-files as the command-line arguments. Another commend-line argument required is the path to your API information. The following is an example of command-line for executing the code, using the 7th sub-file.
```
python TDX_retriever.py your_api_info.json 7
```
The following block is an example of the layout in the api info json file.

In fact, we execute the code using batch script files (".bat" files). As we just mentioned, there are multiple subfiles and such batch files makes rerun the code much easier. Before an example of the content in the batch file, here are some notes:
* Things after "@REM" are comments.
* In our case, we set the batch file name as "run_py_key1.bat" and read the api information from a json file called "api_key1.json".
* We put the serial number as a command-line argument here, and the batch file will pass this number to the step we execute the code.

Batch file commmand-line execution example: 
```
run_py_key1.bat 7
```
The following code is an example of batch files.

In [ ]:
@REM set the file name as a variable, for example, "fname" would be "run_py_key1"
set fname=%~n0
@REM %fname:~7,4% is extracting 4 characters,
@REM starting from the 8th character from "run_py_key1"
@REM therefore "%fname:~7,4%" would be "key1"
python TDX_retriever.py env\api_%fname:~7,4%.json %1

Since we use VS code as our main editor, there's a function called "task" that makes our lives even easier when it comes to executing batch files. To do this, we need to create a file called "tasks.json" and save it in a folder called ".vscode". Example of the "tasks.json" file as the next block.

# 2. Fill Missing 